In [7]:
# Pilar Fernández Gayol
# Illinois Institute of Technology 
# ETS Ingenieros Informáticos, Universidad Politecnica de Madrid


<h1 style="font-size: 3em;">Integration and Tutorial of pmacctd Toolset on FABRIC Testbed</h1>

#  Introduction
The FABRIC testbed is a central hub for large-scale networking experimentation and research, offering a platform for exploring innovative networking technologies. pmacct, a set of open-source passive monitoring tools by Paolo Lucente, is widely used for network traffic monitoring. Integrating pmacct into FABRIC enhances its monitoring capabilities, enabling real-time traffic analysis in an advanced research setting. 

This project leverages multiple technologies to develop and execute experiments, integrating pmacct into FABRIC to advance network monitoring and experimentation. The goal is to provide real-world tools and settings for computer science and engineering students to explore and innovate in networking.

> Throughout this notebook we will show the integration of pmacctd in FABRIC, we will perform several experiments that show the multiple capabilities offered by this tool.

<div style="text-align: center;">
    <img src="./images/1.png" width="350px" alt="pmacctd integration on a FABRIC node"/>
</div>



# Setup and Installation

This section includes the necessary installations and configurations required to run the subsequent code. It ensures that all dependencies and environments are correctly set up.

## Creation of the slice

For this experiments we are going to create a slice named "pmacctd_slice" with two nodes (n1 and n2), each with specified resources, connected by a layer 2 network (L2Bridge). The architecture enables experimentation with network configurations and monitoring.

Each node uses an Ubuntu 20.04 image, which allows the installation of the pmacctd tool on it. It also provides a robust and flexible operating system for experimenting with different network configurations and monitoring setups.

<div style="text-align: center;">
    <img src="./images/2.png" width="400px"/>
</div>

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Initialize the FABRIC library manager
fablib = fablib_manager()

# Define the slice and node names
slice_name = "pmacctd_slice"
n1_name = "n1"
n2_name = "n2"

# Try to find an existing slice with the given name
slice = None
for slice_candidate in fablib.get_slices():
    if slice_candidate.get_name() == slice_name:
        slice = slice_candidate
        break

# If the slice does not exist, create a new one
if None == slice:
    image = 'default_ubuntu_20'
    site = fablib.get_random_sites(count=1)[0]
    slice = fablib.new_slice(name=slice_name)

    # Add the first node to the slice with specified resources
    n1 = slice.add_node(name=n1_name, image=image, cores=2, ram=4, disk=50, site=site)
    n1_iface1 = n1.add_component(model="NIC_Basic", name="iface1").get_interfaces()[0]

    # Add the second node to the slice with specified resources
    n2 = slice.add_node(name=n2_name, image=image, cores=2, ram=4, disk=50, site=site)
    n2_iface1 = n2.add_component(model="NIC_Basic", name="iface1").get_interfaces()[0]

    # Create a layer 2 network connecting both nodes
    net = slice.add_l2network(name="net", interfaces=[n1_iface1, n2_iface1], type="L2Bridge")
    slice.submit()

# Retrieve the nodes from the slice
n1 = slice.get_node(name=n1_name)
n2 = slice.get_node(name=n2_name)

# Display the slice information
slice.show()


ID,ee639505-2555-443c-a5f8-0c3a15891d31
Name,pmacctd_slice
Lease Expiration (UTC),2024-07-18 08:47:24 +0000
Lease Start (UTC),2024-07-04 08:47:26 +0000
Project ID,6bd91382-d009-4f0b-8269-158816ee381d
State,StableOK


ID,ee639505-2555-443c-a5f8-0c3a15891d31
Name,pmacctd_slice
Lease Expiration (UTC),2024-07-18 08:47:24 +0000
Lease Start (UTC),2024-07-04 08:47:26 +0000
Project ID,6bd91382-d009-4f0b-8269-158816ee381d
State,StableOK


## Configuration of the slice


##### Installing Essential Development and Network Tools on FABRIC Nodes
   
The following code updates the package lists and installs essential development tools (build-essential), network utilities (net-tools), and the Python Scapy library (python3-scapy) on the two nodes (n1 and n2).
These tools are necessary for compiling software, managing network configurations, and manipulating network packets, preparing the nodes for network experiments and monitoring tasks of pmacctd.



In [27]:

n1.execute("sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy")
n2.execute("sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy")
   

36 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

  

 Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
 

 Reading package lists...
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
 

 Reading package lists...
Building dependency tree...
Reading state information...
python3-scapy is already the newest version (2.4.3-4).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

  

 Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgr

("36 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nbuild-essential is already the newest version (12.8ubuntu1.1).\n0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nnet-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).\n0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\npython3-scapy is already the newest version (2.4.3-4).\n0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.\n",
 '\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.\n\n\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.\n\n\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.\n\n\nWARNING: apt does

##### Testing SSH Connectivity and Retrieving Commands for FABRIC Nodes
   
This code tests the SSH connectivity on the two nodes (n1 and n2), printing whether SSH works on each node. 
It also retrieves and prints the SSH command for connecting to each node, these commands can be pasted on a terminal to gain access to the node ubuntu terminal for each node.

If the result is False, an error has occurred during the creation of the slice; please delete it and try to create it again.

In [3]:
print("ssh works on n1: " + str(n1.test_ssh()))
print(n1.get_ssh_command())

print("ssh works on n2: " + str(n2.test_ssh()))
print(n2.get_ssh_command())

ssh works on n1: True
ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:febc:8269
ssh works on n2: True
ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feb1:d79f


##### Displaying Network Interfaces on FABRIC Nodes
   
This code prints the network configuration of two nodes (n1 and n2) by executing the ifconfig command on each node.

In [4]:
# Management vs Experiment interfaces.
print("ifconfig n1:")
n1.execute("ifconfig")
print("ifconfig n2:")
n2.execute("ifconfig")

ifconfig n1:
enp3s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.30.6.165  netmask 255.255.254.0  broadcast 10.30.7.255
        inet6 fe80::f816:3eff:febc:8269  prefixlen 64  scopeid 0x20<link>
        inet6 2001:400:a100:3000:f816:3eff:febc:8269  prefixlen 64  scopeid 0x0<global>
        ether fa:16:3e:bc:82:69  txqueuelen 1000  (Ethernet)
        RX packets 650664  bytes 475685911 (475.6 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 108623  bytes 16304050 (16.3 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

enp7s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet6 fd3f:f209:c712::1  prefixlen 48  scopeid 0x0<global>
        inet6 fe80::48d:91ff:fed1:234f  prefixlen 64  scopeid 0x20<link>
        ether 06:8d:91:d1:23:4f  txqueuelen 1000  (Ethernet)
        RX packets 26229097  bytes 39681313769 (39.6 GB)
        RX errors 0  dropped 2073  overruns 0  frame 0
        TX packets 280828  bytes

('enp3s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000\n        inet 10.30.6.20  netmask 255.255.254.0  broadcast 10.30.7.255\n        inet6 fe80::f816:3eff:feb1:d79f  prefixlen 64  scopeid 0x20<link>\n        inet6 2001:400:a100:3000:f816:3eff:feb1:d79f  prefixlen 64  scopeid 0x0<global>\n        ether fa:16:3e:b1:d7:9f  txqueuelen 1000  (Ethernet)\n        RX packets 629599  bytes 463579420 (463.5 MB)\n        RX errors 0  dropped 0  overruns 0  frame 0\n        TX packets 88886  bytes 10270452 (10.2 MB)\n        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0\n\nenp7s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500\n        inet6 fd3f:f209:c712::2  prefixlen 48  scopeid 0x0<global>\n        inet6 fe80::820:c4ff:fe71:4439  prefixlen 64  scopeid 0x20<link>\n        ether 0a:20:c4:71:44:39  txqueuelen 1000  (Ethernet)\n        RX packets 280579  bytes 24152207 (24.1 MB)\n        RX errors 0  dropped 0  overruns 0  frame 0\n        TX packets 26231417  bytes 39

##### Configuring IPv6 Addresses on the Nodes
This code configures IPv6 addresses for the interfaces on the two nodes (n1 and n2). It assigns local IPv6 addresses and then displays the interface configurations.

In [5]:
from ipaddress import IPv6Address, IPv6Network

n1_iface1 = n1.get_interface(name = "n1-iface1-p1")
n2_iface1 = n2.get_interface(name = "n2-iface1-p1")

n1_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c712::1"), subnet = IPv6Network("fd3f:f209:c712::/48"))
n2_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c712::2"), subnet = IPv6Network("fd3f:f209:c712::/48"))
n1_iface1.show()
n2_iface1.show()

Name,n1-iface1-p1
Short Name,p1
Node,n1
Network,net
Bandwidth,100
Mode,config
VLAN,
MAC,06:8D:91:D1:23:4F
Physical Device,enp7s0
Device,enp7s0
IP Address,fd3f:f209:c712::1


Name,n2-iface1-p1
Short Name,p1
Node,n2
Network,net
Bandwidth,100
Mode,config
VLAN,
MAC,0A:20:C4:71:44:39
Physical Device,enp7s0
Device,enp7s0
IP Address,fd3f:f209:c712::2


Name,n2-iface1-p1
Short Name,p1
Node,n2
Network,net
Bandwidth,100
Mode,config
VLAN,
MAC,0A:20:C4:71:44:39
Physical Device,enp7s0
Device,enp7s0
IP Address,fd3f:f209:c712::2


##### Configuring the Network Interface
This code configurates the network interfaces of the two nodes (n1 and n2), bringing up the network interfaces using the ifconfig command with root priviledges.

In [6]:
n1.execute("sudo ifconfig " + n1_iface1.get_device_name() + " up")
n2.execute("sudo ifconfig " + n2_iface1.get_device_name() + " up")

('', '')

##### IPv6 Ping Test Between Nodes
The following cell performs a single ICMPv6 ping test between network interfaces of nodes n1 and n2. 
The code executes these commands on both nodes to verify IPv6 connectivity between them.

In [7]:
command = "ping -c 1 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::2"
print(command)
n1.execute(command) 

command2 = "ping -c 1 -I " + n2_iface1.get_device_name() + " fd3f:f209:c712::1"
print(command2)
n2.execute(command2)

ping -c 1 -I enp7s0 fd3f:f209:c712::2
PING fd3f:f209:c712::2(fd3f:f209:c712::2) from fd3f:f209:c712::1 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::2: icmp_seq=1 ttl=64 time=0.188 ms

--- fd3f:f209:c712::2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.188/0.188/0.188/0.000 ms
ping -c 1 -I enp7s0 fd3f:f209:c712::1
PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.144 ms

--- fd3f:f209:c712::1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.144/0.144/0.144/0.000 ms


('PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes\n64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.144 ms\n\n--- fd3f:f209:c712::1 ping statistics ---\n1 packets transmitted, 1 received, 0% packet loss, time 0ms\nrtt min/avg/max/mdev = 0.144/0.144/0.144/0.000 ms\n',
 '')

##### Transfering files to the nodes
The following cell defines two functions that will be subsequently used on the notebook to upload and download files between the JypyterHub workspace and the nodes.

For performing each action it is necessary to place the file on the workspace, and specify the node and the file (its route may be added too).

You just need to run this cell once to use the defined functions.

In [8]:
# Transferring files between your JupyterHub and your node

# NOTE could use n1.upload_file(...)
def cmd_upload_file_to(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/"
    scp_pre_command.insert(5, filename)
    return ' '.join(scp_pre_command)

def cmd_download_file_from(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/" + filename
    scp_pre_command.append(".")
    return ' '.join(scp_pre_command)

## Installation

This section includes the necessary installations and configurations required to run the pmacctd tool and to perform actions and experiments on it. It also ensures that all dependencies and environments are correctly set up.

First, the zip files of the tool and the jansson library must be added to the workspace so that they can be transferred to the nodes.

> Note: You can find the zip files on:
>> pmacctd: http://www.pmacct.net/#downloads
>> jansson library: https://digip.org/jansson/releases/


> Note: You may need to change the names of the files in the cell 

In [10]:
#Upload the zip files to the nodes, you may paste the commands on a terminal to upload the files to the nodes.
print(cmd_upload_file_to(n1, "pmacct-1.7.8.zip"))
print(cmd_upload_file_to(n1, "jansson-2.13.tar.gz"))
print(cmd_upload_file_to(n2, "pmacct-1.7.8.zip"))
print(cmd_upload_file_to(n2, "jansson-2.13.tar.gz"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config pmacct-1.7.8.zip ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config jansson-2.13.tar.gz ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config pmacct-1.7.8.zip ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config jansson-2.13.tar.gz ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


> The following code unzips the files loaded to the nodes, but first installs the neccessary software packages.


In [11]:
# Unzip both files
# First, install zip in the nodes
n1.execute("sudo apt-get install -y zip")
n1.execute("unzip pmacct-1.7.8.zip")
n1.execute("tar -xzf jansson-2.13.tar.gz")

n2.execute("sudo apt-get install -y zip")
n2.execute("unzip pmacct-1.7.8.zip")
n2.execute("tar -xzf jansson-2.13.tar.gz")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 96 not upgraded.
Need to get 336 kB of archives.
After this operation, 1231 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 unzip amd64 6.0-25ubuntu1.2 [169 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 zip amd64 3.0-11build1 [167 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
 Fetched 336 kB in 0s (706 kB/s)
 debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to r

('', '')

##### Jansson library installation
The following code installs the jansson library on the nodes. This library is needed so that the pmacctd tool can diplay the information of the results on the experiments in a JSON format.


In [12]:
## Installation of jansson library
n1.execute("cd jansson-2.13 && ./configure && make && sudo make install")
n2.execute("cd jansson-2.13 && ./configure && make && sudo make install")

checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /usr/bin/mkdir -p
checking for gawk... gawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking whether gcc understands -c and -o together... yes
checking whether make supports the include directive... yes (GNU style)
checking dependency style of gcc... gcc3
checking for g++... g++
checking whether we are using the GNU C++ compiler... yes
checking whether g++ accepts -g... yes
checking de

('checking for a BSD-compatible install... /usr/bin/install -c\nchecking whether build environment is sane... yes\nchecking for a thread-safe mkdir -p... /usr/bin/mkdir -p\nchecking for gawk... gawk\nchecking whether make sets $(MAKE)... yes\nchecking whether make supports nested variables... yes\nchecking for gcc... gcc\nchecking whether the C compiler works... yes\nchecking for C compiler default output file name... a.out\nchecking for suffix of executables... \nchecking whether we are cross compiling... no\nchecking for suffix of object files... o\nchecking whether we are using the GNU C compiler... yes\nchecking whether gcc accepts -g... yes\nchecking for gcc option to accept ISO C89... none needed\nchecking whether gcc understands -c and -o together... yes\nchecking whether make supports the include directive... yes (GNU style)\nchecking dependency style of gcc... gcc3\nchecking for g++... g++\nchecking whether we are using the GNU C++ compiler... yes\nchecking whether g++ accepts

##### pmacctd installation 
The following code installs the neccesary dependencies for the use of the tool and then installs it following the commands.

1. Place on the pmacct folder (cd pmacct-1.7.8)
2. Run the command ./autogen.sh
3. Run the command ./configure --enable-jansson (so that the results can be created on a JSON format too)
4. Run the command make
5. Run the command make install (superuser privileges)


In [13]:
## Installation of pmacctd 

n1.execute("sudo apt-get install -y libpcap-dev pkg-config libtool autoconf automake make bash libstdc++6 g++ libjansson4")
n1.execute("cd pmacct-1.7.8 && ./autogen.sh && ./configure --enable-jansson && make && sudo make install")

n2.execute("sudo apt-get install -y libpcap-dev pkg-config libtool autoconf automake make bash libstdc++6 g++ libjansson4")
n2.execute("cd pmacct-1.7.8 && ./autogen.sh && ./configure --enable-jansson && make && sudo make install")

Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:9.3.0-1ubuntu2).
g++ set to manually installed.
make is already the newest version (4.2.1-1.2).
make set to manually installed.
bash is already the newest version (5.0-6ubuntu1.2).
libstdc++6 is already the newest version (10.5.0-1ubuntu1~20.04).
libstdc++6 set to manually installed.
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc gettext libtool-doc gfortran
  | fortran95-compiler gcj-jdk m4-doc
The following NEW packages will be installed:
  autoconf automake autotools-dev libjansson4 libltdl-dev libpcap-dev
  libpcap0.8-dev libtool m4 pkg-config
0 upgraded, 10 newly installed, 0 to remove and 96 not upgraded.
Need to get 1727 kB of archives.
After this operation, 7306 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 m4 amd64 1.4.18-4 [199 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ub

("autogen.sh: reconfigure with autoreconf\nlibtoolize: putting auxiliary files in '.'.\nlibtoolize: copying file './ltmain.sh'\nlibtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'm4'.\nlibtoolize: copying file 'm4/libtool.m4'\nlibtoolize: copying file 'm4/ltoptions.m4'\nlibtoolize: copying file 'm4/ltsugar.m4'\nlibtoolize: copying file 'm4/ltversion.m4'\nlibtoolize: copying file 'm4/lt~obsolete.m4'\nautogen.sh: for the next step, run './configure' [or './configure --help' to check available options]\nchecking for a BSD-compatible install... /usr/bin/install -c\nchecking whether build environment is sane... yes\nchecking for a thread-safe mkdir -p... /usr/bin/mkdir -p\nchecking for gawk... gawk\nchecking whether make sets $(MAKE)... yes\nchecking whether make supports nested variables... yes\nchecking build system type... x86_64-pc-linux-gnu\nchecking host system type... x86_64-pc-linux-gnu\nchecking how to print strings... printf\nchecking whether make supports the include directive.

# Experiments and use of the tool
The following sections describe different experiments performed on the pmacctd tool to know more about its capabilities. The experiments are performed both using .pcap files and live traffic captured by pmacctd.

> You can use any .pcap file but I would be using the following one for the experiments gmail.pcapng.cap, which can be found in: https://packetlife.net/captures/
You need to first upload it to the node using the upload function and then move it to the pmacctd folder: print(cmd_upload_file_to(n1, "gmail.pcapng.cap"))

> We are going to use different configurations of the tool so that it works properly. These configurations can be found on the configuration-files folder and placed on the Jupyter Workspace. You can play with them so that you can learn how the tool works.

## Experiment of the pmacctd tool using a pcap file.
In this section we are going to perform three different experiments using the same pcap file but with different configurations of the tool. We are going to show the results obtained from processing the pmacctd tool on a pcap file in three different formats: CSV, JSON and .txt. 

For the execution of the tool a configuration file is needed, in this section the following fields are necessary: 

daemonize: false \
debug: true \
pcap_savefile: file.pcap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m   


 - daemonize: This field indicates whether the process should run in the background or not. In this case, will set to false, meaning the process will run in the foreground.
  - debug: This field controls the debug mode. When set to true, it enables debugging output, which is be helpful for these first exeperiments.
  - pcap_savefile: This field specifies the path where the tool obtains the data. It will set to $PATH_of_pmacct_tool/pmacct-1.7.8/file.pcap. The file contains captured network traffic data.
  - aggregate: This field specifies the fields by which to aggregate the captured data. In this case, it will aggregate data based on source host, destination host, source port, destination port, and protocol.
    > The output would look like this:
    <div style="text-align: center;">
    <img src="./images/3.png" width="1000px"/>
</div>
  - plugin_buffer_size: This field defines the buffer size used by plugins. 
  - plugins: This field specifies the plugins to be used. In this example, the print plugin is specified, which likely indicates that the captured data will be printed. 
  - print_output: This field specifies the output format for the printed data. It will be set to csv for the first experiment since we want our output in this format.
  - print_history_roundoff: This field specifies the rounding-off interval for historical data. 

### CSV format
The configuration file will have the following data: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_csv.csv \
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_csv.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [14]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_csv.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_csv.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_csv.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_csv.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result csv file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the csv result file into your computer.

In [4]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_csv.conf pmacct-1.7.8")
n2.execute("mv pcap_csv.conf pmacct-1.7.8")
n1.execute("touch result_pcap_csv.csv && mv result_pcap_csv.csv pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_csv.conf && cat result_pcap_csv.csv")

 mv: cannot stat 'pcap_csv.conf': No such file or directory
  mv: cannot stat 'pcap_csv.conf': No such file or directory
 SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTOCOL,PACKETS,BYTES
192.168.1.101,178.123.13.120,42559,26895,udp,1,95
208.117.231.17,192.168.1.101,443,56561,tcp,71,102240
192.168.1.101,208.117.231.17,56562,443,tcp,47,2212
208.117.231.17,192.168.1.101,443,56562,tcp,54,77760
192.168.1.101,208.117.231.17,56561,443,tcp,53,2444
208.117.231.17,192.168.1.101,443,56563,tcp,63,90720
178.123.13.120,192.168.1.101,26895,42559,udp,1,98
192.168.1.101,208.117.231.17,56563,443,tcp,45,2188
93.184.221.133,192.168.1.101,80,56668,tcp,15,20212
192.168.1.100,239.255.255.250,54714,1900,udp,2,322
192.168.1.101,90.84.59.130,56671,443,tcp,4,161
192.168.1.101,93.184.221.133,56668,80,tcp,11,501
192.168.1.101,173.194.35.53,56643,443,tcp,21,5083
90.84.136.33,192.168.1.101,80,56582,tcp,4,5760
173.194.35.53,192.168.1.101,443,56643,tcp,23,13572
192.168.1.101,90.84.136.33,56582,80,tcp,4,196
192.168.1.101,90.84.1

('SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTOCOL,PACKETS,BYTES\n192.168.1.101,178.123.13.120,42559,26895,udp,1,95\n208.117.231.17,192.168.1.101,443,56561,tcp,71,102240\n192.168.1.101,208.117.231.17,56562,443,tcp,47,2212\n208.117.231.17,192.168.1.101,443,56562,tcp,54,77760\n192.168.1.101,208.117.231.17,56561,443,tcp,53,2444\n208.117.231.17,192.168.1.101,443,56563,tcp,63,90720\n178.123.13.120,192.168.1.101,26895,42559,udp,1,98\n192.168.1.101,208.117.231.17,56563,443,tcp,45,2188\n93.184.221.133,192.168.1.101,80,56668,tcp,15,20212\n192.168.1.100,239.255.255.250,54714,1900,udp,2,322\n192.168.1.101,90.84.59.130,56671,443,tcp,4,161\n192.168.1.101,93.184.221.133,56668,80,tcp,11,501\n192.168.1.101,173.194.35.53,56643,443,tcp,21,5083\n90.84.136.33,192.168.1.101,80,56582,tcp,4,5760\n173.194.35.53,192.168.1.101,443,56643,tcp,23,13572\n192.168.1.101,90.84.136.33,56582,80,tcp,4,196\n192.168.1.101,90.84.136.32,56600,80,tcp,1,41\n192.168.1.101,68.232.35.139,56659,443,tcp,3,151\n192.168.1.101,173.194.66.84,5

In [16]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_csv.csv"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_pcap_csv.csv .


### JSON format
The configuration file will have the following data: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_json.json \
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_json.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [17]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_json.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_json.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_json.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_json.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result .json file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the json result file into your computer.

In [18]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_json.conf pmacct-1.7.8")
n2.execute("mv pcap_json.conf pmacct-1.7.8")
n1.execute("touch result_pcap_json.json && mv result_pcap_json.json pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_json.conf && cat result_pcap_json.json")

{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "178.123.13.120", "port_src": 42559, "port_dst": 26895, "ip_proto": "udp", "packets": 1, "bytes": 95}
{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56561, "ip_proto": "tcp", "packets": 71, "bytes": 102240}
{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "208.117.231.17", "port_src": 56562, "port_dst": 443, "ip_proto": "tcp", "packets": 47, "bytes": 2212}
{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56562, "ip_proto": "tcp", "packets": 54, "bytes": 77760}
{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "208.117.231.17", "port_src": 56561, "port_dst": 443, "ip_proto": "tcp", "packets": 53, "bytes": 2444}
{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56563, "ip_proto": "tcp", "packets": 63, "bytes": 90720}
{"event_type"

('{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "178.123.13.120", "port_src": 42559, "port_dst": 26895, "ip_proto": "udp", "packets": 1, "bytes": 95}\n{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56561, "ip_proto": "tcp", "packets": 71, "bytes": 102240}\n{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "208.117.231.17", "port_src": 56562, "port_dst": 443, "ip_proto": "tcp", "packets": 47, "bytes": 2212}\n{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56562, "ip_proto": "tcp", "packets": 54, "bytes": 77760}\n{"event_type": "purge", "ip_src": "192.168.1.101", "ip_dst": "208.117.231.17", "port_src": 56561, "port_dst": 443, "ip_proto": "tcp", "packets": 53, "bytes": 2444}\n{"event_type": "purge", "ip_src": "208.117.231.17", "ip_dst": "192.168.1.101", "port_src": 443, "port_dst": 56563, "ip_proto": "tcp", "packets": 63, "bytes": 90720}\n{"eve

In [19]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_json.json"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_pcap_json.json .


### TXT Format



The configuration file will have the following data: \

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: formatted \
print_output_file: result_pcap_formatted.txt\
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_formatted.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.


In [20]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_formatted.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_formatted.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_formatted.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_formatted.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result txt file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .txt result file into your computer.

In [21]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_formatted.conf pmacct-1.7.8")
n2.execute("mv pcap_formatted.conf pmacct-1.7.8")
n1.execute("touch result_pcap_formatted.txt && mv result_pcap_formatted.txt pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_formatted.conf && cat result_pcap_formatted.txt")

SRC_IP                                         DST_IP                                         SRC_PORT  DST_PORT  PROTOCOL    PACKETS               BYTES
192.168.1.101                                  178.123.13.120                                 42559     26895     udp         1                     95
208.117.231.17                                 192.168.1.101                                  443       56561     tcp         71                    102240
192.168.1.101                                  208.117.231.17                                 56562     443       tcp         47                    2212
208.117.231.17                                 192.168.1.101                                  443       56562     tcp         54                    77760
192.168.1.101                                  208.117.231.17                                 56561     443       tcp         53                    2444
208.117.231.17                                 192.168.1.101                    

('SRC_IP                                         DST_IP                                         SRC_PORT  DST_PORT  PROTOCOL    PACKETS               BYTES\n192.168.1.101                                  178.123.13.120                                 42559     26895     udp         1                     95\n208.117.231.17                                 192.168.1.101                                  443       56561     tcp         71                    102240\n192.168.1.101                                  208.117.231.17                                 56562     443       tcp         47                    2212\n208.117.231.17                                 192.168.1.101                                  443       56562     tcp         54                    77760\n192.168.1.101                                  208.117.231.17                                 56561     443       tcp         53                    2444\n208.117.231.17                                 192.168.1.101            

In [22]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_formatted.txt"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_pcap_formatted.txt .


### Customization 

Pmacctd offers an important characteristic, which is that enables the user to choose which fields wants to obtain from the captured data. This can be done using the "aggregate" field on the configuration file.

All the available aggregate fields are: src_mac, dst_mac, vlan, in_vlan, out_vlan, cos, etype, src_host, dst_host,src_net, dst_net, src_mask, dst_mask, src_as,    dst_as, src_port, dst_port, tos, proto, none, sum_mac, sum_host, sum_net, sum_as, sum_port, flows, tag, tag2, label, class, tcpflags, in_iface, out_iface, std_comm, ext_comm, lrg_comm, as_path, peer_src_ip, peer_dst_ip, peer_src_as, peer_dst_as, local_pref, med, dst_roa, src_std_comm, src_ext_comm, src_lrg_comm, src_as_path, src_local_pref, src_med, src_roa, mpls_vpn_rd, mpls_pw_id, mpls_label_top, mpls_label_bottom, mpls_label_stack, sampling_rate, sampling_direction, src_host_country, dst_host_country, src_host_pocode, dst_host_pocode, src_host_coords, dst_host_coords, nat_event, fw_event, post_nat_src_host, post_nat_dst_host, post_nat_src_port, post_nat_dst_port, tunnel_src_mac, tunnel_dst_mac, tunnel_src_host, tunnel_dst_host, tunnel_proto, tunnel_tos, tunnel_src_port, tunnel_dst_port, tunnel_tcpflags, fwd_status, vxlan, timestamp_start, timestamp_end, timestamp_arrival, timestamp_export, export_proto_seqno, export_proto_version, export_proto_sysid.

Some of them may not be supported by some packets but we can make a small example to test some of them (the most used ones).
 
daemonize: false \
debug: true \
pcap_interface: any \
aggregate: src_host, dst_host, src_port, dst_port, proto, src_mac, dst_mac, etype, in_iface, out_iface, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: formatted \
print_output_file: result_custom.txt \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file live_custom.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [43]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_custom.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_custom.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_custom.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_custom.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result txt file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .txt result file into your computer.

In [44]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_custom.txt")
n1.execute("mv pcap_custom.conf result_custom.txt pmacct-1.7.8")
n2.execute("mv pcap_custom.conf pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_custom.conf && cat result_custom.txt")

 mv: cannot stat 'pcap_custom.conf': No such file or directory
  mv: cannot stat 'pcap_custom.conf': No such file or directory
 SRC_MAC            DST_MAC            ETYPE  IN_IFACE    OUT_IFACE   SRC_IP                                         DST_IP                                         SRC_PORT  DST_PORT  PROTOCOL    PACKETS               BYTES
00:14:0b:33:33:27  d0:7a:b5:96:cd:0a  800    0           0           192.168.1.101                                  178.123.13.120                                 42559     26895     udp         1                     95
d0:7a:b5:96:cd:0a  00:14:0b:33:33:27  800    0           0           208.117.231.17                                 192.168.1.101                                  443       56561     tcp         71                    102240
00:14:0b:33:33:27  d0:7a:b5:96:cd:0a  800    0           0           192.168.1.101                                  208.117.231.17                                 56562     443       tcp         47        

('SRC_MAC            DST_MAC            ETYPE  IN_IFACE    OUT_IFACE   SRC_IP                                         DST_IP                                         SRC_PORT  DST_PORT  PROTOCOL    PACKETS               BYTES\n00:14:0b:33:33:27  d0:7a:b5:96:cd:0a  800    0           0           192.168.1.101                                  178.123.13.120                                 42559     26895     udp         1                     95\nd0:7a:b5:96:cd:0a  00:14:0b:33:33:27  800    0           0           208.117.231.17                                 192.168.1.101                                  443       56561     tcp         71                    102240\n00:14:0b:33:33:27  d0:7a:b5:96:cd:0a  800    0           0           192.168.1.101                                  208.117.231.17                                 56562     443       tcp         47                    2212\nd0:7a:b5:96:cd:0a  00:14:0b:33:33:27  800    0           0           208.117.231.17                      

In [25]:
#Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_custom.txt"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_custom.txt .


### Filtering data
Pmacctd allow filtering the data captured using the pcap_filter field on the configuration file.

This filter must be written as in tcpdump, in order to work properly. The following configuration file shows an example of it.
 
daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
pcap_filter: tcp \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_filter.csv \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file pcap_filter.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

The expected output is a csv file which only contains information about tcp packets.

In [18]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_filter.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_filter.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_filter.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/pcap_filter.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result csv file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .csv result file into your computer.

In [21]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_pcap_filter.csv")
n1.execute("mv pcap_filter.conf result_pcap_filter.csv pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_filter.conf && cat result_pcap_filter.csv")

 mv: cannot stat 'pcap_filter.conf': No such file or directory
 SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTOCOL,PACKETS,BYTES
208.117.231.17,192.168.1.101,443,56561,tcp,71,102240
192.168.1.101,208.117.231.17,56562,443,tcp,47,2212
208.117.231.17,192.168.1.101,443,56562,tcp,54,77760
192.168.1.101,208.117.231.17,56561,443,tcp,53,2444
208.117.231.17,192.168.1.101,443,56563,tcp,63,90720
192.168.1.101,208.117.231.17,56563,443,tcp,45,2188
93.184.221.133,192.168.1.101,80,56668,tcp,15,20212
192.168.1.101,90.84.59.130,56671,443,tcp,4,161
192.168.1.101,93.184.221.133,56668,80,tcp,11,501
192.168.1.101,173.194.35.53,56643,443,tcp,21,5083
90.84.136.33,192.168.1.101,80,56582,tcp,4,5760
173.194.35.53,192.168.1.101,443,56643,tcp,23,13572
192.168.1.101,90.84.136.33,56582,80,tcp,4,196
192.168.1.101,90.84.136.32,56600,80,tcp,1,41
192.168.1.101,68.232.35.139,56659,443,tcp,3,151
192.168.1.101,173.194.66.84,56651,443,tcp,5,2114
68.232.35.139,192.168.1.101,443,56659,tcp,4,191
173.194.66.84,192.168.1.101,443,56651,tc

('SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTOCOL,PACKETS,BYTES\n208.117.231.17,192.168.1.101,443,56561,tcp,71,102240\n192.168.1.101,208.117.231.17,56562,443,tcp,47,2212\n208.117.231.17,192.168.1.101,443,56562,tcp,54,77760\n192.168.1.101,208.117.231.17,56561,443,tcp,53,2444\n208.117.231.17,192.168.1.101,443,56563,tcp,63,90720\n192.168.1.101,208.117.231.17,56563,443,tcp,45,2188\n93.184.221.133,192.168.1.101,80,56668,tcp,15,20212\n192.168.1.101,90.84.59.130,56671,443,tcp,4,161\n192.168.1.101,93.184.221.133,56668,80,tcp,11,501\n192.168.1.101,173.194.35.53,56643,443,tcp,21,5083\n90.84.136.33,192.168.1.101,80,56582,tcp,4,5760\n173.194.35.53,192.168.1.101,443,56643,tcp,23,13572\n192.168.1.101,90.84.136.33,56582,80,tcp,4,196\n192.168.1.101,90.84.136.32,56600,80,tcp,1,41\n192.168.1.101,68.232.35.139,56659,443,tcp,3,151\n192.168.1.101,173.194.66.84,56651,443,tcp,5,2114\n68.232.35.139,192.168.1.101,443,56659,tcp,4,191\n173.194.66.84,192.168.1.101,443,56651,tcp,7,4599\n208.117.231.17,192.168.1.101,443,5

In [22]:
#Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_filter.csv"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_pcap_filter.csv .


## Experiment of the pmacctd tool using live traffic
In this section we are going to perform an experiment using live traffic sending ICMP packets between nodes by using the "ping" command. We are going to show the results obtained from processing the pmacctd tool on a file in the formatted different format. This format can be changed to any different one as we did on the last section, you just have to change its value on the configuration file and re-uploaded it to the node. 

For the execution of the tool a configuration file is needed, in this section the forllowing fields are necessary: 

daemonize: false \
debug: true \
pcap_interface: any/eth3s0 \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m   

  - pcap_interface: This field specifies the interface where the tool captures the data from. The value can be "any" or the name of the interface itself.
    
For this, we are going to upload to the node the following configuration file live.conf which contains the information shown.

In [23]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/live.conf"))
print(cmd_upload_file_to(n2, "configuration-files/live.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/live.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/live.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .txt result file into your computer.

In [40]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_live.txt")
n1.execute("mv live.conf result_live.txt pmacct-1.7.8")
n2.execute("mv live.conf pmacct-1.7.8")

 mv: cannot stat 'live.conf': No such file or directory
  mv: cannot stat 'live.conf': No such file or directory
 

('', "mv: cannot stat 'live.conf': No such file or directory\n")

##### Concurrent Execution
The following cell uses the Python's threading module for concurrent execution of tasks. Two threads are created to run functions run_pmacctd on nodes n1 and n2, executing commands command_n1 and command_n2 respectively. This allows simultaneous operation of network monitoring (pmacctd) and IPv6 ping testing. The use of join() ensures synchronization, waiting for completion of the second thread before ending execution.

This is used because when using live traffic pmacctd acts as a demon so it needs to be stopped manually. Since we are testing the live traffic capabilities of pmacctd, we can use threads so that first the node n1 starts the daemon of pmacctd and then n2 pings 20 times to n1. Then when the second command finishes also the pmacctd tool stops so that we can see the results.

In [41]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f live.conf"
command_n2 = "ping -c 20 fd3f:f209:c712::1"

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.095 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=2 ttl=64 time=0.106 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=3 ttl=64 time=0.114 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=4 ttl=64 time=0.077 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=5 ttl=64 time=0.121 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=6 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=7 ttl=64 time=0.075 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=8 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=9 ttl=64 time=0.075 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=10 ttl=64 time=0.102 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=11 ttl=64 time=0.100 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=12 ttl=64 time=0.079 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=13 ttl=64 time=0.103 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=14 ttl=64 time=0.102 ms
64 bytes from fd3f:f209:c712:

In [42]:
#Show results
n1.execute("cd pmacct-1.7.8 && cat result_live.txt")
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_live.txt"))

SRC_IP                                         DST_IP                                         SRC_PORT  DST_PORT  PROTOCOL    PACKETS               BYTES
fe80::f816:3eff:febb:9db7                      2001:400:a100:3000:f816:3eff:febc:8269         0         0         ipv6-icmp   2                     144
2001:400:a100:3000:f816:3eff:febc:8269         fe80::f816:3eff:febb:9db7                      0         0         ipv6-icmp   2                     128
2600:2701:5000:a902::c                         2001:400:a100:3000:f816:3eff:febc:8269         47776     22        tcp         176                   16140
2001:400:a100:3000:f816:3eff:febc:8269         2600:2701:5000:a902::c                         22        47776     tcp         109                   26668
2001:400:a100:3000:f816:3eff:febc:8269         2600:2701:5000:a902::c                         22        33070     tcp         2                     416
2001:400:a100:3000:f816:3eff:febc:8269         2600:2701:5000:a902::c             

### Filtering data
The pmacctd tool allows for data filtering through the pcap_filter field in its configuration file. This feature is particularly useful when working with live traffic, as it enables you to focus on specific data of interest or identify anomalies. By applying filters, you can streamline your data analysis, making it easier to pinpoint relevant information or troubleshoot issues.

This filter must be written as in tcpdump, in order to work properly. The following configuration file shows an example of it. This time we are going to implement a mmore complex filter. We are going to filter by the ip of the host and the type protocol. We can also delete that information from the aggregation field since all the data collected will present the same data of them. 
 
daemonize: false \
debug: true \
pcap_interface: any \
pcap_filter: icmp6 and src host fd3f:f209:c712::2 \
aggregate: src_port, dst_port, etype, in_iface, out_iface, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_live_filter.csv \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file live_filter.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

The expected output is a csv file which only contains information about icmp packets with source IP address fd3f:f209:c712::2 (n2).

In [9]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/live_filter.conf"))
print(cmd_upload_file_to(n2, "configuration-files/live_filter.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/live_filter.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/live_filter.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .csv result file into your computer.

In [10]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_live_filter.csv")
n1.execute("mv live_filter.conf result_live_filter.csv pmacct-1.7.8")

('', '')

##### Concurrent Execution
The following cell uses the Python's threading module for concurrent execution of tasks. Two threads are created to run functions run_pmacctd on nodes n1 and n2, executing commands command_n1 and command_n2 respectively. This allows simultaneous operation of network monitoring (pmacctd) and IPv6 ping testing. The use of join() ensures synchronization, waiting for completion of the second thread before ending execution.

This is used because when using live traffic pmacctd acts as a demon so it needs to be stopped manually. Since we are testing the live traffic capabilities of pmacctd, we can use threads so that first the node n1 starts the daemon of pmacctd and then n2 pings 20 times to n1. Then when the second command finishes also the pmacctd tool stops so that we can see the results.

In [11]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f live_filter.conf"
command_n2 = "ping -c 20 fd3f:f209:c712::1"

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.123 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=2 ttl=64 time=0.136 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=3 ttl=64 time=0.084 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=4 ttl=64 time=0.084 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=5 ttl=64 time=0.177 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=6 ttl=64 time=0.118 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=7 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=8 ttl=64 time=0.100 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=9 ttl=64 time=0.082 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=10 ttl=64 time=0.114 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=11 ttl=64 time=0.115 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=12 ttl=64 time=0.122 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=13 ttl=64 time=0.080 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=14 ttl=64 time=0.094 ms
64 bytes from fd3f:f209:c712:

In [12]:
# Show results
n1.execute("cd pmacct-1.7.8 && cat result_live_filter.csv")

ETYPE,IN_IFACE,OUT_IFACE,SRC_PORT,DST_PORT,PACKETS,BYTES
86dd,0,0,0,0,17,1728


('ETYPE,IN_IFACE,OUT_IFACE,SRC_PORT,DST_PORT,PACKETS,BYTES\n86dd,0,0,0,0,17,1728\n',
 '')

In [13]:
# Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_live_filter.csv"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/result_live_filter.csv .


## Experiment of the pmacctd tool using live traffic with iPerf

In this section we are going to continue to use live traffic to test the pmacctd tool, but this time we are going to use iPerf. 
iPerf is a network performance testing tool used to measure bandwidth between two hosts over a TCP or UDP connection. iPerf works by establishing a client-server model where one instance of iPerf acts as the server and another as the client. 

The objective of this is to test the pmacctd tool this time with UDP and TCP packets, instead of ICMP ones. 

For the execution of the tool a configuration file is needed, in this section the following fields are necessary: 

daemonize: false \
debug: true \
pcap_interface: any \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file iperf.conf which contains the information shown.

In [28]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/iperf.conf"))
print(cmd_upload_file_to(n2, "configuration-files/iperf.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/iperf.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/iperf.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


> We need to install iperf on the nodes, since we use IPV6 we will use iperf3, which supports it.

In [29]:
#Install iperf3
n1.execute("sudo apt-get install -y iperf3")
n2.execute("sudo apt-get install -y iperf3")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libiperf0 libsctp1
Suggested packages:
  lksctp-tools
The following NEW packages will be installed:
  iperf3 libiperf0 libsctp1
0 upgraded, 3 newly installed, 0 to remove and 96 not upgraded.
Need to get 94.1 kB of archives.
After this operation, 331 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libsctp1 amd64 1.0.18+dfsg-1 [7876 B]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 libiperf0 amd64 3.7-3 [72.0 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 iperf3 amd64 3.7-3 [14.2 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readli

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  libiperf0 libsctp1\nSuggested packages:\n  lksctp-tools\nThe following NEW packages will be installed:\n  iperf3 libiperf0 libsctp1\n0 upgraded, 3 newly installed, 0 to remove and 96 not upgraded.\nNeed to get 94.1 kB of archives.\nAfter this operation, 331 kB of additional disk space will be used.\nGet:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libsctp1 amd64 1.0.18+dfsg-1 [7876 B]\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 libiperf0 amd64 3.7-3 [72.0 kB]\nGet:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 iperf3 amd64 3.7-3 [14.2 kB]\nFetched 94.1 kB in 1s (143 kB/s)\nSelecting previously unselected package libsctp1:amd64.\r\n(Reading database ... \r(Reading database ... 5%\r(Reading database ... 10%\r(Reading database ... 15%\r(Reading database ... 20%\r(Reading database 

##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .txt result file into your computer.

In [13]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_iperf.csv")
n1.execute("mv iperf.conf result_iperf.csv pmacct-1.7.8")
n2.execute("mv iperf.conf pmacct-1.7.8")

 mv: cannot stat 'iperf.conf': No such file or directory
  mv: cannot stat 'iperf.conf': No such file or directory
 

('', "mv: cannot stat 'iperf.conf': No such file or directory\n")

##### iPerf

This time we are going to use the same thread approach to test the tool, but using iPerf. n1 will act as the server node and n2 wil act as the client, but first we need to execute the daemon tool on the server node, which will finish when the last command finishes.

> Note: It must work, but if not, you will need to look for the PID of the process by getting into the terminal of the node and put the command ps aux | grep iperf and then use the command sudo kill -9 "PID" so that the address gets free and you can execute the iPerf again.

In [14]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command = "cd pmacct-1.7.8 && sudo pmacctd -f iperf.conf"
command_n1 = "iperf3 -s"
command_n2 = "iperf3 -c fd3f:f209:c712::1 -6"

thread = threading.Thread(target=run_iperf, args=(n1, command))
thread_n1 = threading.Thread(target=run_iperf, args=(n1, command_n1))
thread_n2 = threading.Thread(target=run_iperf, args=(n2, command_n2))

thread.start()
thread_n1.start()
thread_n2.start()

thread_n2.join()

 iperf3: error - unable to start listener for connections: Address already in use
iperf3: exiting
 ('', 'iperf3: error - unable to start listener for connections: Address already in use\niperf3: exiting\n')
Connecting to host fd3f:f209:c712::1, port 5201
[  5] local fd3f:f209:c712::2 port 51486 connected to fd3f:f209:c712::1 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  1.48 GBytes  12.7 Gbits/sec  173   1.48 MBytes       
[  5]   1.00-2.00   sec  1.40 GBytes  12.0 Gbits/sec    0   1.52 MBytes       
[  5]   2.00-3.00   sec  1.42 GBytes  12.2 Gbits/sec    0   1.52 MBytes       
[  5]   3.00-4.00   sec  1.44 GBytes  12.4 Gbits/sec    9   1.07 MBytes       
[  5]   4.00-5.00   sec  1.42 GBytes  12.2 Gbits/sec    0   1.37 MBytes       
[  5]   5.00-6.00   sec  1.28 GBytes  11.0 Gbits/sec   35   1.10 MBytes       
[  5]   6.00-7.00   sec  1.29 GBytes  11.1 Gbits/sec    0   1.29 MBytes       
[  5]   7.00-8.00   sec  1.44 GBytes  12.4 Gb

In [32]:
#Show results
n1.execute("cd pmacct-1.7.8 && cat result_iperf.csv")


SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTOCOL,PACKETS,BYTES
fe80::f816:3eff:fe95:17ba,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe1b:80fc,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe4d:d1ec,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fef8:383d,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe60:613f,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe58:aabb,ff02::16,0,0,ipv6-icmp,2,152
fe80::f816:3eff:fe33:b599,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe90:cf9,ff02::16,0,0,ipv6-icmp,3,228
fe80::f816:3eff:fe2e:9d7a,ff02::16,0,0,ipv6-icmp,2,152
fe80::f816:3eff:fe84:4d18,ff02::16,0,0,ipv6-icmp,3,228
2001:400:a100:3000:f816:3eff:febc:8269,2610:1e0:1700:202::7,22,43360,tcp,2,416
2001:400:a100:3000:f816:3eff:febc:8269,2610:1e0:1700:202::7,22,46976,tcp,2,416
2001:400:a100:3000:f816:3eff:febc:8269,2600:2701:5000:a902::c,22,34822,tcp,2,416
2610:1e0:1700:202::7,2001:400:a100:3000:f816:3eff:febc:8269,46976,22,tcp,2,144
2610:1e0:1700:202::7,2001:400:a100:3000:f816:3eff:febc:8269,43360,22,tcp,2,144

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_iperf.csv"))

## Experiment of the pmacctd tool using a pcap file with IPFIX flows

IPFIX (IP Flow Information Export) flows are a standard method for exporting detailed information about IP traffic flows from network devices like routers and switches. They capture metadata including packet counts, timestamps, protocol types, ports, source and destination IP addresses, and so on. Security, traffic analysis, and network monitoring are all done with this data. IPFIX assists in anomaly detection and performance optimization. It is also a more advanced and flexible version of NetFlow, allowing for more detailed and customizable flow information export.

We are going to first work producing IPFIX flows with pmacctd using a pcap file, this will include first print the result on a terminal, then on a log file at pmacctd then we are going to create an .ipfix file which will be opened and analyzed with tshark and lastly we are going to send the flows to a collector using the Netflow Python library.

### Print the result on the terminal

For this first and easiest experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe, print \
nfprobe_version: 10 

- nfprobe: This plugin is used for exporting network flow data using the NetFlow/IPFIX protocol.
- print: This plugin is used for printing the collected data, in this case to standard output since we have not specified an output file.
- nfprobe_version: 10 refers to IPFIX, which is also known as NetFlow version 10. 

In [18]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_1.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_1.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_1.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_1.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute pmacctd.
3. Show the results.

In [19]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv ipfix_1.conf pmacct-1.7.8")
n2.execute("mv ipfix_1.conf pmacct-1.7.8")

('', '')

In [20]:
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f ipfix_1.conf")

SRC_IP                                         PACKETS               BYTES
192.168.1.101                                  366                   30583
208.117.231.17                                 239                   344160
178.123.13.120                                 1                     98
93.184.221.133                                 15                    20212
192.168.1.100                                  2                     322
90.84.136.33                                   9                     6020
173.194.35.53                                  26                    13704
68.232.35.139                                  4                     191
173.194.66.84                                  11                    4881
157.56.106.184                                 1                     137
fe80::6d49:f772:ed9b:d16a                      3                     283
74.125.206.125                                 1                     66
173.194.35.39                           

('SRC_IP                                         PACKETS               BYTES\n192.168.1.101                                  366                   30583\n208.117.231.17                                 239                   344160\n178.123.13.120                                 1                     98\n93.184.221.133                                 15                    20212\n192.168.1.100                                  2                     322\n90.84.136.33                                   9                     6020\n173.194.35.53                                  26                    13704\n68.232.35.139                                  4                     191\n173.194.66.84                                  11                    4881\n157.56.106.184                                 1                     137\nfe80::6d49:f772:ed9b:d16a                      3                     283\n74.125.206.125                                 1                     66\n173.194.35.39            

### Print the result on a log file

For this second experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe, print \
nfprobe_version: 10 \
print_output_file: ipfix_2.log

- print: This plugin is used for printing the collected data, in this case to a log file since we have specified an output file.
- print_output_file: refers to the log output file.

In [21]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_2.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_2.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_2.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_2.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Create the result result log file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .log result file into your computer.

In [22]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_2.log")
n1.execute("mv ipfix_2.conf ipfix_2.log pmacct-1.7.8")
n2.execute("mv ipfix_2.conf pmacct-1.7.8")

('', '')

In [23]:
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f ipfix_2.conf")
n1.execute("cd pmacct-1.7.8 && cat ipfix_2.log")

 INFO ( default/core ): Promiscuous Mode Accounting Daemon, pmacctd 1.7.8-git (RELEASE)
INFO ( default/core ):  '--enable-jansson' '--enable-l2' '--enable-traffic-bins' '--enable-bgp-bins' '--enable-bmp-bins' '--enable-st-bins'
INFO ( default/core ): Reading configuration file '/home/ubuntu/pmacct-1.7.8/ipfix_2.conf'.
WARN ( default_print/print ): defaulting to SRC HOST aggregation.
INFO ( default/core ): [,0] link type is: 1
INFO ( default_nfprobe/nfprobe ): plugin_pipe_size=4096000 bytes plugin_buffer_size=10240 bytes
INFO ( default_nfprobe/nfprobe ): ctrl channel: obtained=212992 bytes target=3200 bytes
INFO ( default_print/print ): plugin_pipe_size=4096000 bytes plugin_buffer_size=10240 bytes
INFO ( default_print/print ): ctrl channel: obtained=212992 bytes target=3200 bytes
INFO ( default_nfprobe/nfprobe ): NetFlow probe plugin is originally based on softflowd 0.9.7 software, Copyright 2002 Damien Miller <djm@mindrot.org> All rights reserved.
INFO ( default_nfprobe/nfprobe ):     

('SRC_IP                                         PACKETS               BYTES\n192.168.1.101                                  366                   30583\n208.117.231.17                                 239                   344160\n178.123.13.120                                 1                     98\n93.184.221.133                                 15                    20212\n192.168.1.100                                  2                     322\n90.84.136.33                                   9                     6020\n173.194.35.53                                  26                    13704\n68.232.35.139                                  4                     191\n173.194.66.84                                  11                    4881\n157.56.106.184                                 1                     137\nfe80::6d49:f772:ed9b:d16a                      3                     283\n74.125.206.125                                 1                     66\n173.194.35.39            

In [24]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_2.log"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/ipfix_2.log .


### Send the IPFIX flows to an .ipfix file 

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.1:4739 \
nfprobe_version: 10 

- nfprobe_receiver: it configures pmacct to send the collected network flow data to a local collector on IP address 127.0.0.1 (local) running on port 4739.

In [38]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_3.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_3.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_3.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_3.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Create the result result IPFIX file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Redirect the network flow of 127.0.0.1 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [39]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_flows_3.ipfix")
n1.execute("mv ipfix_3.conf ipfix_flows_3.ipfix pmacct-1.7.8")
n2.execute("mv ipfix_3.conf pmacct-1.7.8")

('', '')

##### Redirecting the flow of 127.0.0.1 port 4739 to the file. 
We are going to do that using netcat. So we first need to download it and then use the following command to redirect the ipfix flows created to it: nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_3.ipfix.

This command means that we are using the NetCat utility using UDP which listens for incoming connections on port 4739 and which redirects the output to the file specified on the path, in this cas ipfix_flows_3.ipfix


In [40]:
n1.execute("sudo apt-get install netcat")
n2.execute("sudo apt-get install netcat")

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  netcat
0 upgraded, 1 newly installed, 0 to remove and 96 not upgraded.
Need to get 2172 B of archives.
After this operation, 15.4 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 netcat all 1.206-1ubuntu1 [2172 B]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
 Fetched 2172 B in 0s (6732 B/s)
Selecting previously unselected package netcat.
(Reading database ... 72809 files and directories currently installed.)
Preparing to unpack .../netcat_1.2

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following NEW packages will be installed:\n  netcat\n0 upgraded, 1 newly installed, 0 to remove and 96 not upgraded.\nNeed to get 2172 B of archives.\nAfter this operation, 15.4 kB of additional disk space will be used.\nGet:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 netcat all 1.206-1ubuntu1 [2172 B]\nFetched 2172 B in 0s (6794 B/s)\nSelecting previously unselected package netcat.\r\n(Reading database ... \r(Reading database ... 5%\r(Reading database ... 10%\r(Reading database ... 15%\r(Reading database ... 20%\r(Reading database ... 25%\r(Reading database ... 30%\r(Reading database ... 35%\r(Reading database ... 40%\r(Reading database ... 45%\r(Reading database ... 50%\r(Reading database ... 55%\r(Reading database ... 60%\r(Reading database ... 65%\r(Reading database ... 70%\r(Reading database ... 75%\r(Reading database ... 80%\r(Reading database ... 85%\r(Reading datab

In [41]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_3.ipfix"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_3.conf"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))

thread_1.start()
thread_2.start()

thread_2.join()

 INFO ( default/core ): Promiscuous Mode Accounting Daemon, pmacctd 1.7.8-git (RELEASE)
INFO ( default/core ):  '--enable-jansson' '--enable-l2' '--enable-traffic-bins' '--enable-bgp-bins' '--enable-bmp-bins' '--enable-st-bins'
INFO ( default/core ): Reading configuration file '/home/ubuntu/pmacct-1.7.8/ipfix_3.conf'.
INFO ( default/core ): [,0] link type is: 1
INFO ( default_nfprobe/nfprobe ): plugin_pipe_size=4096000 bytes plugin_buffer_size=10240 bytes
INFO ( default_nfprobe/nfprobe ): ctrl channel: obtained=212992 bytes target=3200 bytes
INFO ( default/core ): PCAP capture file, sleeping for 2 seconds
INFO ( default_nfprobe/nfprobe ): NetFlow probe plugin is originally based on softflowd 0.9.7 software, Copyright 2002 Damien Miller <djm@mindrot.org> All rights reserved.
INFO ( default_nfprobe/nfprobe ):           TCP timeout: 3600s
INFO ( default_nfprobe/nfprobe ):  TCP post-RST timeout: 120s
INFO ( default_nfprobe/nfprobe ):  TCP post-FIN timeout: 300s
INFO ( default_nfprobe/nfpro

In [42]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_flows_3.ipfix"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/ipfix_flows_3.ipfix .


##### We need to download t-shark so that we can se the content of the flows in the result file. 
First, we need to download the file to inspect its content and the different flows. Since downloading TShark requires granting some permissions, we cannot automate this process. Please refer to the cell where the SSH command for node n1 is shown in the section. Execute that command in a terminal (so that that terminal is now the terminal of the node), and then run the following command: "sudo apt-get install -y tshark"


In [55]:
#n1.execute("sudo apt-get install -y tshark")
#n2.execute("sudo apt-get install -y tshark")

 E: Could not get lock /var/lib/dpkg/lock-frontend. It is held by process 74499 (apt-get)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), is another process using it?
  E: Could not get lock /var/lib/dpkg/lock-frontend. It is held by process 72208 (apt-get)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), is another process using it?
 

('',
 'E: Could not get lock /var/lib/dpkg/lock-frontend. It is held by process 72208 (apt-get)\nE: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), is another process using it?\n')

##### Summary
The following command will read and display the packet summary information from the IPFIX file, showing a list of packets captured in the file.


In [43]:
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_3.ipfix")

    1          0              →              CFLOW 408 IPFIX flow ( 408 bytes) Obs-Domain-ID=    0 [Data-Template:1024] [Data-Template:1025] [Data-Template:2048] [Data-Template:2049] [Data:1024]
    2          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]
    3          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]
    4          0              →              CFLOW 488 IPFIX flow ( 488 bytes) Obs-Domain-ID=    0 [Data:1024] [Data:2048] [Data:1024] [Data:2048]
    5          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]
    6          0              →              CFLOW 508 IPFIX flow ( 508 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]
    7          0              →              CFLOW 452 IPFIX flow ( 452 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]
    8          0              →              CFLOW 476 IPFIX flow (

('    1          0              →              CFLOW 408 IPFIX flow ( 408 bytes) Obs-Domain-ID=    0 [Data-Template:1024] [Data-Template:1025] [Data-Template:2048] [Data-Template:2049] [Data:1024]\n    2          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]\n    3          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]\n    4          0              →              CFLOW 488 IPFIX flow ( 488 bytes) Obs-Domain-ID=    0 [Data:1024] [Data:2048] [Data:1024] [Data:2048]\n    5          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]\n    6          0              →              CFLOW 508 IPFIX flow ( 508 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]\n    7          0              →              CFLOW 452 IPFIX flow ( 452 bytes) Obs-Domain-ID=    0 [Data:2048] [Data:1024]\n    8          0              →              CFLOW 476 IPF

##### In detail 
This command reads the IPFIX file and provides a detailed hex and ASCII representation of the first packet's data. This is useful for in-depth analysis of the packet's structure and content.

You can change the number of the packet and execute the command again to inspect different ones.


In [45]:
# Look one packet of a flow in detail
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_3.ipfix -x -c 1")

0000  00 0a 01 98 66 86 6c a6 00 00 00 00 00 00 00 00   ....f.l.........
0010  00 02 00 44 04 00 00 0f 00 99 00 08 00 98 00 08   ...D............
0020  00 01 00 08 00 02 00 08 00 3c 00 01 00 0a 00 04   .........<......
0030  00 0e 00 04 00 3d 00 01 00 08 00 04 00 0c 00 04   .....=..........
0040  00 07 00 02 00 0b 00 02 00 05 00 01 00 06 00 01   ................
0050  00 04 00 01 00 02 00 44 04 01 00 0f 00 99 00 08   .......D........
0060  00 98 00 08 00 01 00 08 00 02 00 08 00 3c 00 01   .............<..
0070  00 0a 00 04 00 0e 00 04 00 3d 00 01 00 08 00 04   .........=......
0080  00 0c 00 04 00 07 00 02 00 0b 00 02 00 05 00 01   ................
0090  00 06 00 01 00 04 00 01 00 02 00 44 08 00 00 0f   ...........D....
00a0  00 99 00 08 00 98 00 08 00 01 00 08 00 02 00 08   ................
00b0  00 3c 00 01 00 0a 00 04 00 0e 00 04 00 3d 00 01   .<...........=..
00c0  00 1b 00 10 00 1c 00 10 00 05 00 01 00 07 00 02   ................
00d0  00 0b 00 02 00 06 00 01 00 04 00 01 00 02 00 

('0000  00 0a 01 98 66 86 6c a6 00 00 00 00 00 00 00 00   ....f.l.........\n0010  00 02 00 44 04 00 00 0f 00 99 00 08 00 98 00 08   ...D............\n0020  00 01 00 08 00 02 00 08 00 3c 00 01 00 0a 00 04   .........<......\n0030  00 0e 00 04 00 3d 00 01 00 08 00 04 00 0c 00 04   .....=..........\n0040  00 07 00 02 00 0b 00 02 00 05 00 01 00 06 00 01   ................\n0050  00 04 00 01 00 02 00 44 04 01 00 0f 00 99 00 08   .......D........\n0060  00 98 00 08 00 01 00 08 00 02 00 08 00 3c 00 01   .............<..\n0070  00 0a 00 04 00 0e 00 04 00 3d 00 01 00 08 00 04   .........=......\n0080  00 0c 00 04 00 07 00 02 00 0b 00 02 00 05 00 01   ................\n0090  00 06 00 01 00 04 00 01 00 02 00 44 08 00 00 0f   ...........D....\n00a0  00 99 00 08 00 98 00 08 00 01 00 08 00 02 00 08   ................\n00b0  00 3c 00 01 00 0a 00 04 00 0e 00 04 00 3d 00 01   .<...........=..\n00c0  00 1b 00 10 00 1c 00 10 00 05 00 01 00 07 00 02   ................\n00d0  00 0b 00 02 00 06 00 01 00 04 

### Send the IPFIX flows to a collector

First, we need to download the netflow Python library from  https://pypi.org/project/netflow/ and place on the work directory. 
Also it is necessary to download python3 on the node: sudo apt-get install python3 to be able to use the library. Then we will uploade it to the nodes. 

In [47]:
# Please, place the resulting command on a terminal
print(cmd_upload_file_to(n1, "netflow-0.12.2.tar.gz"))
print(cmd_upload_file_to(n2, "netflow-0.12.2.tar.gz"))      

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config netflow-0.12.2.tar.gz ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config netflow-0.12.2.tar.gz ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


In [48]:
# Unzip the netflow library file, you can install the zip library by executing the command sudo apt-get zip, you can uncomment the following line and execute it
# If you follow this notebook from the beggining you should have the software package already installed.
#n1.execute("sudo apt-get zip")
n1.execute("tar -xzf netflow-0.12.2.tar.gz")
n2.execute("tar -xzf netflow-0.12.2.tar.gz")

('', '')

It is necessary to download python3 and pip on the node to be able to use the library.

In [49]:
# Download pyython at the nodes and also netflow
n1.execute("sudo apt-get install -y python3")
n2.execute("sudo apt-get install -y python3")
n1.execute("sudo apt-get install -y pip")
n2.execute("sudo apt-get install -y pip")

Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.8.2-0ubuntu2).
python3 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.8.2-0ubuntu2).
python3 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev
  python3-wheel python3.8-dev zlib1g-dev
The following NEW packages will be installed:
  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev
  python3-pip python3-wheel python3.8-dev zlib1g-dev
0 upgraded, 9 newly installed, 0 to remove and 96 not upgraded.
Need to get 6804 kB of archives.
Afte

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev\n  python3-wheel python3.8-dev zlib1g-dev\nThe following NEW packages will be installed:\n  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev\n  python3-pip python3-wheel python3.8-dev zlib1g-dev\n0 upgraded, 9 newly installed, 0 to remove and 96 not upgraded.\nNeed to get 6804 kB of archives.\nAfter this operation, 25.6 MB of additional disk space will be used.\nGet:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 libexpat1-dev amd64 2.2.9-1ubuntu0.6 [116 kB]\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-dev amd64 3.8.10-0ubuntu1~20.04.9 [3950 kB]\nGet:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libpython3-dev amd64 3.8.2-0ubuntu2 [7236 B]\nGet:4 http://nova.clouds

We also need to install the netflow Python package so that the library works properly.

In [50]:
n1.execute("pip install netflow")

('Collecting netflow\n  Downloading netflow-0.12.2-py3-none-any.whl (34 kB)\nInstalling collected packages: netflow\nSuccessfully installed netflow-0.12.2\n',
 '')

##### Configuration file

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.2:4739 \
nfprobe_version: 9 

- nfprobe_version: 9 refers to NetFlow, since we are using its Python version we need to specify it.

In [51]:
# Please, place the resulting command on a terminal
print(cmd_upload_file_to(n1, "configuration-files/ipfix_4.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_4.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_4.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_4.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute the python program collector.py specifying the host and the port 
3. Redirect the network flow of 127.0.0.2 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [53]:
n1.execute("mv ipfix_4.conf pmacct-1.7.8")
n2.execute("mv ipfix_4.conf pmacct-1.7.8")

 mv: cannot stat 'ipfix_4.conf': No such file or directory
  mv: cannot stat 'ipfix_4.conf': No such file or directory
 

('', "mv: cannot stat 'ipfix_4.conf': No such file or directory\n")

In [54]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "cd netflow-0.12.2/netflow && python3 collector.py --host 127.0.0.2 --port 4739"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_4.conf"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))

thread_1.start()
thread_2.start()

thread_2.join()

 INFO ( default/core ): Promiscuous Mode Accounting Daemon, pmacctd 1.7.8-git (RELEASE)
INFO ( default/core ):  '--enable-jansson' '--enable-l2' '--enable-traffic-bins' '--enable-bgp-bins' '--enable-bmp-bins' '--enable-st-bins'
INFO ( default/core ): Reading configuration file '/home/ubuntu/pmacct-1.7.8/ipfix_4.conf'.
INFO ( default/core ): [,0] link type is: 1
INFO ( default_nfprobe/nfprobe ): plugin_pipe_size=4096000 bytes plugin_buffer_size=10240 bytes
INFO ( default_nfprobe/nfprobe ): ctrl channel: obtained=212992 bytes target=3200 bytes
INFO ( default/core ): PCAP capture file, sleeping for 2 seconds
INFO ( default_nfprobe/nfprobe ): NetFlow probe plugin is originally based on softflowd 0.9.7 software, Copyright 2002 Damien Miller <djm@mindrot.org> All rights reserved.
INFO ( default_nfprobe/nfprobe ):           TCP timeout: 3600s
INFO ( default_nfprobe/nfprobe ):  TCP post-RST timeout: 120s
INFO ( default_nfprobe/nfprobe ):  TCP post-FIN timeout: 300s
INFO ( default_nfprobe/nfpro

##### Results

The results will be obtained on a .tar file, containing a JSON file placed on the netflow folder as you can see on the following picture:

<div style="text-align: center;">
    <img src="./images/4.png" width="700px"/>
</div>

You can unzip them using the command: gzip -d filename.gz

You can see the results using the command: cat filename

You can download the results using the command "print(cmd_download_file_from(n1, "filename"))" on a new cell of this Notebook 


##  Experiment of the pmacctd tool using live traffic with IPFIX flows
Now we are going to work producing IPFIX flows with pmacctd using the live traffic captured by pmacctd, this will include first printing the result on a terminal, then in a log file at pmacctd then we are going to create an .ipfix file which will be opened and analyzed with tshark and lastly we are going to send the flows to a collector using the Netflow Python library.

### Print the result on the terminal
For this first and easiest experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe, print \
nfprobe_version: 10 \
nfprobe_receiver: localhost:4739

- nfprobe: This plugin is used for exporting network flow data using the NetFlow/IPFIX protocol.
- print: This plugin is used for printing the collected data, in this case to standard output since we have not specified an output file.
- nfprobe_version: 10 refers to IPFIX, which is also known as NetFlow version 10.
- pcap_interface: refers to the interface or interfaces where the tool will collect the data from.
- nfprobe_receiver: specifies the destination to which NetFlow/IPFIX data should be sent.

In [55]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_1.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_1.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_1.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_1.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Start the pmacctd daemon using threads. 
3. Execute ICMP live traffic to the n1 so that it can be collected.
4. Show the results on the terminal.

In [56]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv ipfix_live_1.conf pmacct-1.7.8")
n2.execute("mv ipfix_live_1.conf pmacct-1.7.8")

('', '')

In [57]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_1.conf"
command_n2 = "ping -c 20 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::1"

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.115 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=2 ttl=64 time=0.115 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=3 ttl=64 time=0.067 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=4 ttl=64 time=0.078 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=5 ttl=64 time=0.634 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=6 ttl=64 time=0.068 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=7 ttl=64 time=0.061 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=8 ttl=64 time=0.061 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=9 ttl=64 time=0.060 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=10 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=11 ttl=64 time=0.071 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=12 ttl=64 time=0.064 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=13 ttl=64 time=0.062 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=14 ttl=64 time=0.072 m

### Print the result on a log file

For this second experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe, print \
nfprobe_version: 10 \
print_output_file: ipfix_live_2.log

- print: This plugin is used for printing the collected data, in this case to a log file since we have specified an output file.
- pcap_interface: refers to the interface or interfaces where the tool will collect the data from.
- print_output_file: refers to the log output file.

In [58]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_2.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_2.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_2.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_2.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Create the result result log file.
2. Move the configuration file and the result log file to the pmacctd folder.
3. Execute pmacctd.
4. Create live traffic betweeen the nodes.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [59]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_live_2.log")
n1.execute("mv ipfix_live_2.conf ipfix_live_2.log pmacct-1.7.8")
n2.execute("mv ipfix_live_2.conf pmacct-1.7.8")

('', '')

In [60]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_2.conf"
command_n2 = "ping -c 20 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::1"

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.089 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=2 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=3 ttl=64 time=0.061 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=4 ttl=64 time=0.074 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=5 ttl=64 time=0.112 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=6 ttl=64 time=0.088 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=7 ttl=64 time=0.069 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=8 ttl=64 time=0.068 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=9 ttl=64 time=0.064 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=10 ttl=64 time=0.083 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=11 ttl=64 time=0.060 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=12 ttl=64 time=0.069 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=13 ttl=64 time=0.068 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=14 ttl=64 time=0.098 m

In [61]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_live_2.log"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/ipfix_live_2.log .


### Send the IPFIX flows to an .ipfix file 
For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.4:4738 \
nfprobe_version: 10 

- nfprobe_receiver: it configures pmacct to send the collected network flow data to a local collector on IP address 127.0.0.4 running on port 4739.

In [71]:
# Please, paste the resulting commands to upload the configuration files to the nodes

print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_3.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_3.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_3.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_3.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Redirecting the flow of 127.0.0.4 port 4739 to the file. 
We are going to do that using netcat. So we first need to download it and then use the following command to redirect the ipfix flows created to it: nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_live_3.ipfix.

This command means that we are using the NetCat utility using UDP which listens for incoming connections on port 4739 and which redirects the output to the file specified on the path, in this cas ipfix_flows_3.ipfix


In [63]:
n1.execute("sudo apt-get install -y netcat")
n2.execute("sudo apt-get install -y netcat")

Reading package lists...
Building dependency tree...
Reading state information...
netcat is already the newest version (1.206-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
netcat is already the newest version (1.206-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.


('Reading package lists...\nBuilding dependency tree...\nReading state information...\nnetcat is already the newest version (1.206-1ubuntu1).\n0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.\n',
 '')

##### Execution

1. Create the result result IPFIX file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Redirect the network flow of 127.0.0.4 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [72]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_flows_live_3.ipfix")
n1.execute("mv ipfix_live_3.conf ipfix_flows_live_3.ipfix pmacct-1.7.8")
n2.execute("mv ipfix_live_3.conf pmacct-1.7.8")

('', '')

In [73]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "nc -u -l 4738 > pmacct-1.7.8/ipfix_flows_live_3.ipfix"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_3.conf"
command_3 = "ping -c 20 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::1"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))
thread_3 = threading.Thread(target=run_iperf, args=(n2, command_3))

thread_1.start()
thread_2.start()
thread_3.start()

thread_3.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.117 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=2 ttl=64 time=0.118 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=3 ttl=64 time=0.072 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=4 ttl=64 time=0.083 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=5 ttl=64 time=0.133 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=6 ttl=64 time=0.068 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=7 ttl=64 time=0.073 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=8 ttl=64 time=0.079 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=9 ttl=64 time=0.092 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=10 ttl=64 time=0.074 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=11 ttl=64 time=0.081 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=12 ttl=64 time=0.066 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=13 ttl=64 time=0.067 ms
64 bytes from fd3f:f209:c712::1: icmp_seq=14 ttl=64 time=0.073 m

In [74]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_flows_live_3.ipfix"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/pmacct-1.7.8/ipfix_flows_live_3.ipfix .


##### We need to download t-shark so that we can se the content of the flows in the result file. 
First, we need to download the file to inspect its content and the different flows. Since downloading TShark requires granting some permissions, we cannot automate this process. Please refer to the cell where the SSH command for node n1 is shown in the section. Execute that command in a terminal, and then run the following command: "sudo apt-get install -y tshark"


##### Summary
The following command will read and display the packet summary information from the IPFIX file, showing a list of packets captured in the file.


In [81]:
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_live_3.ipfix")

    1          0              →              CFLOW 464 IPFIX flow ( 464 bytes) Obs-Domain-ID=    0 [Data-Template:1024] [Data-Template:1025] [Data-Template:2048] [Data-Template:2049] [Data:1024]
    2          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]


('    1          0              →              CFLOW 464 IPFIX flow ( 464 bytes) Obs-Domain-ID=    0 [Data-Template:1024] [Data-Template:1025] [Data-Template:2048] [Data-Template:2049] [Data:1024]\n    2          0              →              CFLOW 476 IPFIX flow ( 476 bytes) Obs-Domain-ID=    0 [Data:1024]\n',
 '')

##### In detail 
This command reads the IPFIX file and provides a detailed hex and ASCII representation of the first packet's data. This is useful for in-depth analysis of the packet's structure and content.

You can change the number of the packet and execute the command again to inspect different ones.


In [76]:
# Look one packet of a flow in detai
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_live_3.ipfix -x -c 1")

('', '')

### Send the IPFIX flows to a collector

First, we need to download the netflow Python library from  https://pypi.org/project/netflow/ and place on the work directory. 
Also it is necessary to download python3 on the node: sudo apt-get install python3 to be able to use the library. Then we will uploade it to the nodes. 

In [20]:
# Upload the netflow library to the nodes.
print(cmd_upload_file_to(n1, "netflow-0.12.2.tar.gz"))
print(cmd_upload_file_to(n2, "netflow-0.12.2.tar.gz"))      

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config netflow-0.12.2.tar.gz ubuntu@[2001:48e8:6401:3:f816:3eff:fe5b:5ef4]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config netflow-0.12.2.tar.gz ubuntu@[2001:48e8:6401:3:f816:3eff:fef8:a33f]:~/


In [21]:
# Unzip the netflow library file, you can install the zip library by executing the command sudo apt-get zip, you can uncomment the following line and execute it
# If you follow this notebook from the beggining you should have the software package already installed.
#n1.execute("sudo apt-get zip")
n1.execute("tar -xzf netflow-0.12.2.tar.gz")
n2.execute("tar -xzf netflow-0.12.2.tar.gz")

('', '')

It is necessary to download python3 and pip on the node to be able to use the library.

In [22]:
# Download pyython at the nodes and also netflow
n1.execute("sudo apt-get install -y python3")
n2.execute("sudo apt-get install -y python3")
n1.execute("sudo apt-get install -y pip")
n2.execute("sudo apt-get install -y pip")

Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.8.2-0ubuntu2).
python3 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.8.2-0ubuntu2).
python3 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 96 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev
  python3-wheel python3.8-dev zlib1g-dev
The following NEW packages will be installed:
  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev
  python3-pip python3-wheel python3.8-dev zlib1g-dev
0 upgraded, 9 newly installed, 0 to remove and 96 not upgraded.
Need to get 6804 kB of archives.
Afte

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev\n  python3-wheel python3.8-dev zlib1g-dev\nThe following NEW packages will be installed:\n  libexpat1-dev libpython3-dev libpython3.8-dev python-pip-whl python3-dev\n  python3-pip python3-wheel python3.8-dev zlib1g-dev\n0 upgraded, 9 newly installed, 0 to remove and 96 not upgraded.\nNeed to get 6804 kB of archives.\nAfter this operation, 25.6 MB of additional disk space will be used.\nGet:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 libexpat1-dev amd64 2.2.9-1ubuntu0.6 [116 kB]\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-dev amd64 3.8.10-0ubuntu1~20.04.9 [3950 kB]\nGet:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libpython3-dev amd64 3.8.2-0ubuntu2 [7236 B]\nGet:4 http://nova.clouds

We also need to install the netflow Python package so that the library works properly.

In [25]:
n1.execute("pip install netflow")

('Requirement already satisfied: netflow in ./.local/lib/python3.8/site-packages (0.12.2)\n',
 '')

##### Configuration file

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.6:4739 \
nfprobe_version: 9 

- nfprobe_version: 9 refers to NetFlow, since we are using its Python version we need to specify it.

In [26]:
# Please, paste the commands on the terminal to upload the files to the node
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_4.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_4.conf"))

scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_4.conf ubuntu@[2001:400:a100:3000:f816:3eff:febc:8269]:~/
scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config configuration-files/ipfix_live_4.conf ubuntu@[2001:400:a100:3000:f816:3eff:feb1:d79f]:~/


##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute the python program collector.py specifying the host and the port 
3. Redirect the network flow of 127.0.0.6 port 4739 to the file
4. Execute pmacctd daemon.
5. Create ICMP live traffic
6. Show the results.

In [27]:
n1.execute("mv ipfix_live_4.conf pmacct-1.7.8")
n2.execute("mv ipfix_live_4.conf pmacct-1.7.8")

('', '')

In [28]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "cd netflow-0.12.2/netflow && python3 collector.py --host 127.0.0.6 --port 4739"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_4.conf"
command_3 = "ping -c 40 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::1"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))
thread_3 = threading.Thread(target=run_iperf, args=(n2, command_3))

thread_1.start()
thread_2.start()
thread_3.start()

thread_3.join()

PING fd3f:f209:c712::1(fd3f:f209:c712::1) from fd3f:f209:c712::2 enp7s0: 56 data bytes
64 bytes from fd3f:f209:c712::1: icmp_seq=1 ttl=64 time=0.120 ms
 Traceback (most recent call last):
  File "collector.py", line 231, in <module>
    for ts, client, export in get_export_packets(args.host, args.port):
  File "collector.py", line 182, in get_export_packets
    listener = ThreadedNetFlowListener(host, port)
  File "collector.py", line 95, in __init__
    self.server = QueuingUDPListener((host, port), self.input)
  File "collector.py", line 60, in __init__
    super().__init__(interface, QueuingRequestHandler)
  File "/usr/lib/python3.8/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.8/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use
 ('', 'Traceback (most recent call last):\n  File "collector.py", line 231, in <module>\n    for ts, client, export in get_export_packets(

##### Results

The results will be obtained on a .tar file, containing a JSON file placed on the netflow folder as you can see on the following picture:

<div style="text-align: center;">
    <img src="./images/4.png" width="700px"/>
</div>

You can unzip them using the command: gzip -d filename.gz

You can see the results using the command: cat filename

You can download the results using the command: print(cmd_download_file_from(n1, "filename"))


# Future work
This notebook focuses on the integration of pmacctd with FABRIC, but this is not its ultimate goal. The aim is to create an opportunity for future students and individuals interested in networking within the field of computer science to experiment and learn by leveraging the combined capabilities of these two tools. Therefore, I encourage users to continue contributing to this notebook by modifying experiments, tweaking the pmacctd configuration files, experimenting with different topologies, incorporating pmacctd into their own projects, or simply exploring the numerous functionalities that pmacct offers.